# Reranking Methods in RAG systems
## Overview
Reranking is a crucial step in RAG system that aims to improve the relevance and quality of retrieved documents. It involves reassessing and reordering initially retrieved documents to ensure that the most pertinent information is prioritized for subsequent processing or presentation.
## Motivation
The primary motivation for reranking in RAG systems is to overcome limitations of initial retrieval methods, which often rely on simpler similarity metrics. Reranking allows for more sophisticated relevance assessment, taking into account nuanced relationships between queries and documents that might be msised by traditional retrieval techniques. This process aims to enhance the overall preformance of RAG systems by ensuring that the most relevant information is ues in the generation phase.
## Key Components
Reranking systems typically include the following components:
- Initial Retriever: Often a vector store using embedding-based similarity serach
- Reranking Model: This can be either:
    * LLM for scoring relevance
    * Cross-Encoder model specifically trained for relevance assessment
- Scoring Mechanism: A method to assign relevance scores to documents
- Sorting and Selection Logic: To reorder documents based on new scores
## Benefits of this Approach
Reranking offers several advantages:
- Improved Relevance: By usin gmore sophisticaed models, reranking can capture subtle relevance factors
- Flexibility: Different reranking methods can be applies based on specific needs and resources
- Enhanced Context Quality: Providing more relevant documents to the RAG ssytem improves the quality of generated responses
- Reduced Noise: reranking helps filter out less relevant information, focusing on the most pertinent content
## Conclusion
Reranking is a powerful technique in RAG system that significantly enhances the quality of retrieved information. Whetehr using LLM-based scoring or specialized Cross-Encoder models, reranking allows for more nuanced and accurate assessment of document relevance. This improved relevance translates directly to better performance.
The choice between LLM-based and Cross-Encoder reranking methos depends on factors such as required accuracy, available computational resources, and specific application needs. 

In [3]:
import os
from dotenv import load_dotenv

from langchain_openai.chat_models.azure import AzureChatOpenAI
load_dotenv()
openai_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
openai_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
openai_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_ID")
openai_api_version = os.getenv("AZURE_API_VERSION")

llm = AzureChatOpenAI(
    azure_deployment=openai_deployment,
    api_version="2024-10-01-preview",
    azure_endpoint=f"{openai_endpoint}openai/deployments/{openai_deployment}/chat/completions?api-version=2024-10-01-preview",
    temperature=0,
    logprobs=True,
)

In [1]:
from typing import List, Dict, Any, Tuple
from langchain.chains import RetrievalQA
from langchain_core.retrievers import BaseRetriever
from sentence_transformers import CrossEncoder
path = "./data/Understanding_Climate_Change.pdf"

c:\Users\uidp8109\OneDrive - Continental AG\projects\AdvancedRAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
openai_embedding = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_ID")

from langchain_openai.embeddings.azure import AzureOpenAIEmbeddings
from helper_functions import PyPDFLoader, RecursiveCharacterTextSplitter, replace_t_with_space, FAISS

def encode_pdf(path, chunk_size=1000, chunk_overlap=200):
    """
    Encodes a PDF book into a vector store using OpenAI embeddings.

    Args:
        path: The path to the PDF file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.

    Returns:
        A FAISS vector store containing the encoded book content.
    """

    # Load PDF documents
    loader = PyPDFLoader(path)
    documents = loader.load()


    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len)
    texts = text_splitter.split_documents(documents)

    cleaned_texts = replace_t_with_space(texts)

    embeddings = AzureOpenAIEmbeddings(
        deployment=openai_embedding,
        model="text-embedding-ada-002",
        chunk_size=16
    )
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    return vectorstore

In [5]:
vectorstore = encode_pdf(path)

In [6]:
from pydantic import BaseModel, Field
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
class RatingScore(BaseModel):
    score: float = Field(..., description="he relevance score of a document to a query.")

def rerank_documents(query:str, docs: List[Document], top_n: int=3) -> List[Document]:
    """
    Reranks a list of documents based on their relevance to a query.

    Args:
        query: The query to rerank the documents with.
        docs: The list of documents to rerank.
        top_n: The number of documents to return.

    Returns:
        A list of the top_n documents.
    """
    prompt_template = PromptTemplate(
        input_variables=["query", "doc"],
        template="""On a scale of 1-10, rate the relevance of the following document to the query. Consider the specific context and intent of the query, not just keyword matches.
        Query: {query}
        Document: {doc}
        Relevance Score:"""
    )

    llm_chain = prompt_template | llm.with_structured_output(RatingScore)

    scored_docs = []
    for doc in docs:
        input_data = {"query": query, "doc": doc.page_content}
        score = llm_chain.invoke(input_data).score
        try:
            score = float(score)
        except ValueError:
            score = 0
        scored_docs.append((doc, score))

    reranked_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in reranked_docs[:top_n]]

In [7]:
query = "What are the impacts of climate change on biodiversity?"
initial_docs = vectorstore.similarity_search(query, k=15)
reranked_docs = rerank_documents(query, initial_docs)

# print first 3 initial documents
print("Top initial documents:")
for i, doc in enumerate(initial_docs[:3]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")  # Print first 200 characters of each document


# Print results
print(f"Query: {query}\n")
print("Top reranked documents:")
for i, doc in enumerate(reranked_docs):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")

Top initial documents:

Document 1:
Climate change is altering terrestrial ecosystems by shifting habitat ranges, changing species 
distributions, and impacting ecosystem functions. Forests, grasslands, and deserts are 
experiencing shi...

Document 2:
goals. Policies should promote synergies between biodiversity conservation and climate 
action.  
Chapter 10: Climate Change and Human Health  
Health Impacts  
Heat -Related Illnesses  
Rising temper...

Document 3:
managed retreats.  
Extreme Weather Events  
Climate change is linked to an increase in the frequency and severity of extreme weather 
events, such as hurricanes, heatwaves, droughts, and heavy rainfa...
Query: What are the impacts of climate change on biodiversity?

Top reranked documents:

Document 1:
Climate change is altering terrestrial ecosystems by shifting habitat ranges, changing species 
distributions, and impacting ecosystem functions. Forests, grasslands, and deserts are 
experiencing shi...

Document 2:
Coral re

In [9]:
from typing import Any


from langchain_core.documents import Document


class CustomRetriever(BaseRetriever, BaseModel):
    vectorstore: Any = Field(..., description="Vector store for initial retrieval")

    class Config:
        arbitrary_types_allowed = True

    def _get_relevant_documents(self, query: str, num_doc=2) -> list[Document]:
        initial_docs = self.vectorstore.similarity_search(query, k=5)
        return rerank_documents(query, initial_docs, top_n=num_doc)
    
custom_retriever = CustomRetriever(vectorstore=vectorstore)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=custom_retriever,
    return_source_documents=True,
)

In [11]:
result = qa_chain.invoke({"query": query})

print(f"\nQuestion: {query}")
print(f"Answer: {result['result']}")
print("\nRelevant source documents:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")


Question: What are the impacts of climate change on biodiversity?
Answer: Climate change impacts biodiversity in several significant ways:

1. **Shifting Habitat Ranges**: As temperatures rise, many species are forced to migrate to new areas where the climate is more suitable for their survival. This can lead to changes in the distribution of species across different ecosystems.

2. **Changing Species Distributions**: The alteration in climate conditions can cause some species to expand their range while others may contract or even face extinction if they cannot adapt or migrate.

3. **Impacting Ecosystem Functions**: Changes in species composition can disrupt the balance of ecosystems, affecting functions such as pollination, seed dispersal, and nutrient cycling.

4. **Loss of Biodiversity**: The combined effects of habitat shifts, altered species distributions, and disrupted ecosystem functions can lead to a loss of biodiversity. This loss can weaken ecosystems, making them less res

In [12]:
chunks = [
    "The capital of France is great.",
    "The capital of France is huge.",
    "The capital of France is beautiful.",
    """Have you ever visited Paris? It is a beautiful city where you can eat delicious food and see the Eiffel Tower. 
    I really enjoyed all the cities in france, but its capital with the Eiffel Tower is my favorite city.""", 
    "I really enjoyed my trip to Paris, France. The city is beautiful and the food is delicious. I would love to visit again. Such a great capital city."
]
docs = [Document(page_content=sentence) for sentence in chunks]

In [15]:
from langchain_openai.embeddings import AzureOpenAIEmbeddings
def compare_rag_techniques(query: str, docs: List[Document] = docs) -> None:
    embeddings = AzureOpenAIEmbeddings(
        deployment=openai_embedding,
        model="text-embedding-ada-002",
        chunk_size=16
    )
    vectorstore = FAISS.from_documents(docs, embeddings)

    print("Comparison of Retrieval Techniques")
    print("==================================")
    print(f"Query: {query}\n")
    
    print("Baseline Retrieval Result:")
    baseline_docs = vectorstore.similarity_search(query, k=2)
    for i, doc in enumerate(baseline_docs):
        print(f"\nDocument {i+1}:")
        print(doc.page_content)

    print("\nAdvanced Retrieval Result:")
    custom_retriever = CustomRetriever(vectorstore=vectorstore)
    advanced_docs = custom_retriever._get_relevant_documents(query)
    for i, doc in enumerate(advanced_docs):
        print(f"\nDocument {i+1}:")
        print(doc.page_content)

In [16]:
query = "what is the capital of france?"
compare_rag_techniques(query, docs)

Comparison of Retrieval Techniques
Query: what is the capital of france?

Baseline Retrieval Result:

Document 1:
The capital of France is great.

Document 2:
The capital of France is beautiful.

Advanced Retrieval Result:

Document 1:
The capital of France is great.

Document 2:
I really enjoyed my trip to Paris, France. The city is beautiful and the food is delicious. I would love to visit again. Such a great capital city.


In [17]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

class CrossEncoderRetriever(BaseRetriever, BaseModel):
    vectorstore: Any = Field(description="Vector store for initial retrieval")
    cross_encoder: Any = Field(description="Cross-encoder model for reranking")
    k: int = Field(default=5, description="Number of documents to retrieve initially")
    rerank_top_k: int = Field(default=3, description="Number of documents to return after reranking")

    class Config:
        arbitrary_types_allowed = True

    def _get_relevant_documents(self, query: str, num_doc=2) -> list[Document]:
        initial_docs = self.vectorstore.similarity_search(query, k=self.k)
        
        # Prepare pairs for cross-encoder
        pairs = [[query, doc.page_content] for doc in initial_docs]
        
        # Get cross-encoder scores
        scores = self.cross_encoder.predict(pairs)
        
        # Sort documents by score
        scored_docs = sorted(zip(initial_docs, scores), key=lambda x: x[1], reverse=True)
        
        # Return top reranked documents
        return [doc for doc, _ in scored_docs[:self.rerank_top_k]]
    
    async def aget_relevant_documents(self, query: str) -> List[Document]:
        raise NotImplementedError("Async retrieval not implemented")

c:\Users\uidp8109\OneDrive - Continental AG\projects\AdvancedRAG\venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\uidp8109\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


SSLError: (MaxRetryError("HTTPSConnectionPool(host='cdn-lfs.hf.co', port=443): Max retries exceeded with url: /cross-encoder/ms-marco-MiniLM-L-6-v2/3ae17b87eda3d184502a821fddff43d82feb7c206f665a851c491ec715b497ed?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1731660473&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMTY2MDQ3M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9jcm9zcy1lbmNvZGVyL21zLW1hcmNvLU1pbmlMTS1MLTYtdjIvM2FlMTdiODdlZGEzZDE4NDUwMmE4MjFmZGRmZjQzZDgyZmViN2MyMDZmNjY1YTg1MWM0OTFlYzcxNWI0OTdlZD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVzcG9uc2UtY29udGVudC10eXBlPSoifV19&Signature=uzMg4WZHisw-rDBMcEbYOeDvGTSX5v1yWAu1tcbAXQILx9mCmH4Z1iT7w-edDsHSAodanCzoM2dkZfR15uahrBljAw2UYJQ53jZnIZ002YVXvwW1DzSIv1fWKTuJHOYBupr4WR1IuJECRIZJe5SCcTh2tIUXIPAytjKY8eikEmMD9~bxgVA0HS0Vo8lVwdfsKSRjXysNLaPVkNi5IZZip-xF~NvaqVfB

In [ ]:
query = "What are the impacts of climate change on biodiversity?"
result = qa_chain({"query": query})

print(f"\nQuestion: {query}")
print(f"Answer: {result['result']}")
print("\nRelevant source documents:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")